# Train a Model
### Faster R-CNN
This is just a quick test

In [1]:
import torchvision
from poolDatasetV2 import PoolDatasetV2
import os
import torch
from torchvision.transforms import v2
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader

### Let's start by preparing the data

In [2]:
transforms = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    #v2.RandomHorizontalFlip(p=1),
    v2.ToDtype(torch.float32, scale=True),
    #v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

ROOT_DIR = os.getcwd() + "\\dataset\\images"

In [3]:
ds = PoolDatasetV2(ROOT_DIR,transforms)

In [4]:
print(len(ds))

7097


### Let's get the sizes of the train and test sets

In [5]:
train, test = ds.split_Data()
print(len(train))

4755


In [6]:
print(len(test))

2342


### Let's load it to two different dataloaders

In [7]:
train_dl = DataLoader(train, batch_size=32, shuffle=True)

In [8]:
test_dl = DataLoader(test, batch_size=1171, shuffle=False)

### Let's load our model

In [9]:
model = fasterrcnn_resnet50_fpn(weights = None)

### Now, we need to define the criterion and optimizer
Later, we should use Adam

In [10]:
from torch.optim import SGD
from torch.nn import BCELoss

In [11]:
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

### Finally, let's train!
Training loop:

In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for i, (images, targets) in enumerate(train_dl):
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # Assuming targets are in proper format for Faster R-CNN
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_dl)}")

RuntimeError: each element in list of batch should be of equal size